In [26]:
import pandas as pd
import numpy as np
import matplotlib as plt
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor

In [27]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [28]:
data_train.columns

Index(['id', 'person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_default_on_file',
       'cb_person_cred_hist_length', 'loan_status'],
      dtype='object')

In [29]:
data_train.isnull().sum()

id                            0
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
loan_status                   0
dtype: int64

In [30]:
data_train = data_train.drop('id', axis = 1)
# data_test = data_test.drop('id', axis = 1)

In [31]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58645 entries, 0 to 58644
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  58645 non-null  int64  
 1   person_income               58645 non-null  int64  
 2   person_home_ownership       58645 non-null  object 
 3   person_emp_length           58645 non-null  float64
 4   loan_intent                 58645 non-null  object 
 5   loan_grade                  58645 non-null  object 
 6   loan_amnt                   58645 non-null  int64  
 7   loan_int_rate               58645 non-null  float64
 8   loan_percent_income         58645 non-null  float64
 9   cb_person_default_on_file   58645 non-null  object 
 10  cb_person_cred_hist_length  58645 non-null  int64  
 11  loan_status                 58645 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 5.4+ MB


In [32]:
data_train.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [33]:
data_train['total_intrest'] = data_train['loan_amnt'] * (data_train['loan_int_rate']/100)
data_test['total_intrest'] = data_test['loan_amnt'] * (data_test['loan_int_rate']/100)


In [34]:
data_train['total_intrest'] = data_train['loan_amnt'] + data_train['total_intrest']
data_test['total_intrest'] = data_test['loan_amnt'] + data_test['total_intrest']

In [35]:
data_train = data_train.drop('loan_amnt', axis = 1)
data_train = data_train.drop('loan_int_rate', axis = 1)

data_test = data_test.drop('loan_amnt', axis = 1)
data_test = data_test.drop('loan_int_rate', axis = 1)

In [36]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58645 entries, 0 to 58644
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  58645 non-null  int64  
 1   person_income               58645 non-null  int64  
 2   person_home_ownership       58645 non-null  object 
 3   person_emp_length           58645 non-null  float64
 4   loan_intent                 58645 non-null  object 
 5   loan_grade                  58645 non-null  object 
 6   loan_percent_income         58645 non-null  float64
 7   cb_person_default_on_file   58645 non-null  object 
 8   cb_person_cred_hist_length  58645 non-null  int64  
 9   loan_status                 58645 non-null  int64  
 10  total_intrest               58645 non-null  float64
dtypes: float64(3), int64(4), object(4)
memory usage: 4.9+ MB


In [37]:
from sklearn.preprocessing import LabelEncoder


columns_to_encode = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']


le = LabelEncoder()

for col in columns_to_encode:

    le.fit(data_train[col])
    
    data_train[col] = le.transform(data_train[col])
    data_test[col] = le.transform(data_test[col])


In [38]:
data_train = data_train[['person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length',
       'total_intrest', 'loan_status']]

In [39]:
data_test = data_test[['id','person_age', 'person_income', 'person_home_ownership',
       'person_emp_length', 'loan_intent', 'loan_grade', 'loan_percent_income',
       'cb_person_default_on_file', 'cb_person_cred_hist_length',
       'total_intrest']]

In [40]:
X, y = data_train.to_numpy()[:,:-1], data_train.to_numpy()[:,-1]

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [42]:
model = CatBoostRegressor(iterations=1000, learning_rate=0.5, depth=4, verbose=200)

In [43]:
model.fit(X_train, y_train)

0:	learn: 0.2705825	total: 2.38ms	remaining: 2.38s
200:	learn: 0.1940181	total: 250ms	remaining: 994ms
400:	learn: 0.1867006	total: 488ms	remaining: 729ms
600:	learn: 0.1808400	total: 761ms	remaining: 505ms
800:	learn: 0.1764708	total: 1.02s	remaining: 254ms
999:	learn: 0.1729340	total: 1.28s	remaining: 0us


In [44]:
y_pred = model.predict(X_test)

In [45]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R-squared: {r2:.2f}")

Mean Squared Error: 0.04
R-squared: 0.65


In [46]:
X_test = data_test.iloc[:,1:].to_numpy()

In [47]:
data_test['loan_status'] = model.predict(X_test)

In [48]:
data_test[['id', 'loan_status']].to_csv('submission.csv', index = False)

In [49]:
data_test.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,total_intrest,loan_status
0,58645,23,69000,3,3.0,2,5,0.36,0,2,28940.0,0.750170
1,58646,26,96000,0,6.0,4,2,0.10,1,4,11268.0,-0.001358
2,58647,26,30000,3,5.0,5,4,0.13,1,2,4687.6,0.541515
3,58648,33,50000,3,4.0,0,0,0.14,0,7,7623.0,-0.008678
4,58649,26,102000,0,8.0,2,3,0.15,1,4,17448.0,0.061341


In [50]:
data_test['loan_status']

0        0.750170
1       -0.001358
2        0.541515
3       -0.008678
4        0.061341
           ...   
39093    0.007503
39094   -0.003046
39095   -0.058813
39096    0.293445
39097    0.909809
Name: loan_status, Length: 39098, dtype: float64